### 1.Import Libary

In [1]:
! pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments,Trainer, EarlyStoppingCallback
import numpy as np
import collections
from tqdm.auto import tqdm
from datasets import Dataset, DatasetDict
import evaluate
import wandb

### 2.Data Preprocessing

In [3]:


def read_dataset(file_path):
    """Đọc dataset từ file CSV."""
    df = pd.read_csv(file_path)
    df['context'] = df['context'].astype(str)
    df['question'] = df['question'].astype(str)
    df['answer'] = df['answer'].astype(str)
    return df

def find_start_index(context, answer):
    """Tìm chỉ số bắt đầu của answer trong context."""
    return str(context).find(str(answer))

def prepare_dataset(df):
    TRAIN_RATIO = 0.8
    VAL_RATIO = 0.1
    """Chuẩn bị dataset cho huấn luyện, xác thực và kiểm tra."""
    df['start_index'] = df.apply(lambda row: find_start_index(context=row['context'], answer=row['answer']), axis=1)
    df = df[df['start_index'] != -1]

    dataset_temp = []
    for _, row in df.iterrows():
        sample = {
            'context': row['context'],
            'question': row['question'],
            'answer': {'text': [row['answer']], 'answer_start': [row['start_index']]}
        }
        dataset_temp.append(sample)

    dataset = pd.DataFrame(dataset_temp)

    num_of_total_sample = len(dataset)
    num_of_train_sample = TRAIN_RATIO * num_of_total_sample
    num_of_val_sample = VAL_RATIO * num_of_total_sample

    train_set = dataset.sample(n=int(num_of_train_sample), random_state=42)
    dataset.drop(index=train_set.index, inplace=True)

    val_set = dataset.sample(n=int(num_of_val_sample), random_state=42)
    dataset.drop(index=val_set.index, inplace=True)

    return Dataset.from_pandas(train_set), Dataset.from_pandas(val_set), Dataset.from_pandas(dataset)

def preprocess_training_validation_examples(examples, tokenizer, max_length, stride):
    """Tiền xử lý ví dụ huấn luyện cho mô hình."""
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answer"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

def preprocess_test_examples(examples, tokenizer, max_length, stride):
    """Tiền xử lý ví dụ cho bộ kiểm tra và xác thực."""
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["question"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs


### 3.Fine-Tuning Model

> Function Evaluation

In [4]:

metric = evaluate.load("squad")

def compute_metrics(start_logits, end_logits, features, examples, n_best, max_answer_length, metric):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["question"]
        context = example["context"]
        answers = []

        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["question"], "answers": ex["answer"]} for ex in examples]
    
    for index, (i, ii) in enumerate(zip(predicted_answers, theoretical_answers)):
        if index >= 2:  # Dừng sau 2 mẫu
            break
        print("-" * 99)
        print(f"ID: {i['id']}")
        print(f"Predicted Answer: {i['prediction_text']}")
        print(f"Correct Answers: {', '.join(ii['answers']['text'])}")
        print("-" * 99)
    
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

> Function Load Model and Load tokenizer

In [5]:
def load_model_and_tokenizer(model_checkpoint):
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
    return model, tokenizer

> declare hyperparameters

In [6]:
wandb.login(key = '8a5cbfdaa29778a896996cc679358b1d96cf66b0')

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
MODEL_CHECKPOINT = "google-bert/bert-base-multilingual-cased"
MAX_LENGTH = 512
STRIDE = 380
N_BEST = 180
MAX_ANSWER_LENGTH = 2000

 # Early stopping callback
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=1 ,early_stopping_threshold=0.001)

    # Training arguments
training_args = TrainingArguments(
        output_dir="./bert_question_answer",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=36,
        per_device_eval_batch_size=36,
        gradient_accumulation_steps=12,
        num_train_epochs=15,
        disable_tqdm=False,
        weight_decay=0.25,
        save_total_limit=3,
        optim="adamw_hf",
        fp16=True,
        max_grad_norm=0.6,
        warmup_ratio=0.2,
        group_by_length=True,
        report_to="wandb",
        load_best_model_at_end=True,
        label_names=['start_positions', 'end_positions'],
        lr_scheduler_type="linear"
)


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


> Call functions read dataset

In [8]:
df = read_dataset('/kaggle/input/data-final-legal-full/final_train.csv')

In [9]:
df.shape


(32325, 4)

> call function  prepare_dateset

In [10]:
train_set, val_set, test_set = prepare_dataset(df)

In [11]:
print(f'tập train {len(train_set)}')
print(f'tập val {len(val_set)}')
print(f'test_test{len(test_set)}')

tập train 24368
tập val 3046
test_test3046


> call function Load model and tokenizer

In [12]:
model, tokenizer = load_model_and_tokenizer(MODEL_CHECKPOINT)
model.to('cuda')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

> Embedding 

In [13]:

train_dataset = train_set.map(
        lambda examples: preprocess_training_validation_examples(examples, tokenizer, MAX_LENGTH, STRIDE),
        batched=True,
        remove_columns=train_set.column_names,
    )
val_dataset = val_set.map(
        lambda examples: preprocess_training_validation_examples(examples, tokenizer, MAX_LENGTH, STRIDE),
        batched=True,
        remove_columns=train_set.column_names,
    )
test_dataset = test_set.map(
        lambda examples: preprocess_test_examples(examples, tokenizer, MAX_LENGTH, STRIDE),
        batched=True,
        remove_columns=train_set.column_names,
    )
    

Map:   0%|          | 0/24368 [00:00<?, ? examples/s]

Map:   0%|          | 0/3046 [00:00<?, ? examples/s]

Map:   0%|          | 0/3046 [00:00<?, ? examples/s]

> training process

In [14]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset ,
        callbacks=[early_stopping_callback],
    )
trainer.can_return_loss = True
trainer.train()
    
trainer.save_model(r"./bert_question_answer")


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: hdang1696 (hdang1696-no-work-experience). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /kaggle/working/wandb/ru

Epoch,Training Loss,Validation Loss
0,4.527400,1.435832
1,0.537800,0.173772
2,0.109500,0.085050
4,0.036000,0.049604
5,0.025700,0.047224
6,0.018500,0.047422


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

> Evaluation

In [15]:
predictions, _, _ = trainer.predict(test_dataset)
start_logits, end_logits = predictions    
results = compute_metrics(start_logits, end_logits, test_dataset, test_set, N_BEST, MAX_ANSWER_LENGTH,metric)
print(results)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


  0%|          | 0/3046 [00:00<?, ?it/s]

---------------------------------------------------------------------------------------------------
ID: BHXH bắt buộc và tự nguyện khác nhau thế nào?
Predicted Answer: tự nguyện được quy định tại Khoản 4
Correct Answers: Bảo hiểm xã hội Việt Nam trả lời vấn đề này như sau:Về giải thích từ ngữ, tại Điều 3Luật BHXH năm 2014quy định cụ thể như sau:- BHXH bắt buộc là loại hình BHXH do Nhà nước tổ chức mà người lao động và người sử dụng lao động phải tham gia.- BHXH tự nguyện là loại hình BHXH do Nhà nước tổ chức mà người tham gia được lựa chọn mức đóng, phương thức đóng phù hợp với thu nhập của mình và Nhà nước có chính sách hỗ trợ tiền đóng BHXH để người tham gia hưởng chế độ hưu trí và tử tuất.Về đối tượng áp dụng, tại Điều 2 Luật BHXH năm 2014 quy định, người lao động thuộc đối tượng tham gia BHXH bắt buộc quy định tại Khoản 1 và Khoản 2 bao gồm người lao động là công dân Việt Nam:- Người làm việc theo hợp đồng lao động không xác định thời hạn, hợp đồng lao động xác định thời hạn, hợp đ